In [53]:
!pip install torchvision
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, f1_score
from tqdm import tqdm
import torch
from transformers import AutoTokenizer, AutoModel
from torchvision import models, transforms
from PIL import Image

# Device Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Load and Preprocess Dataset
def load_data(csv_path, image_dir):
    data = pd.read_csv(csv_path)
    images = []
    captions = []
    labels = []

    for idx in range(len(data)):
        img_name = str(data.loc[idx, 'image_id'])
        if not img_name.endswith(".jpg"):
            img_name += ".jpg"
        img_path = os.path.join(image_dir, img_name)

        if os.path.exists(img_path):
            images.append(img_path)
            captions.append(data.loc[idx, 'transcriptions'])
            if 'labels' in data.columns:
                labels.append(int(data.loc[idx, 'labels']))

    return images, captions, labels

# Feature Extraction Functions
def extract_text_features(captions, tokenizer, text_model, max_len=128):
    text_features = []
    for caption in tqdm(captions, desc="Extracting Text Features"):
        inputs = tokenizer(
            caption, return_tensors="pt", truncation=True, padding="max_length", max_length=max_len
        ).to(device)
        with torch.no_grad():
            outputs = text_model(**inputs)
            text_features.append(outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy())
    return np.array(text_features)

def extract_image_features(image_paths, visual_model, transform):
    image_features = []
    for img_path in tqdm(image_paths, desc="Extracting Image Features"):
        image = Image.open(img_path).convert("RGB")
        image = transform(image).unsqueeze(0).to(device)
        with torch.no_grad():
            features = visual_model(image).squeeze().cpu().numpy()
        image_features.append(features)
    return np.array(image_features)

# Training and Evaluation
def train_and_evaluate(X_train, y_train, X_test, y_test, model, model_name):
    print(f"\nTraining {model_name}...")
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    print(f"\nResults for {model_name}:")
    print(classification_report(y_test, preds))
    print("Accuracy:", accuracy_score(y_test, preds))
    print("Macro F1 Score:", f1_score(y_test, preds, average="macro"))
    return model

def main():
    # Paths
    train_csv = "/kaggle/input/tamil-labelled-dataset/Dataset with label/train/train.csv"
    train_images = "/kaggle/input/tamil-labelled-dataset/Dataset with label/train/images"
    test_csv = "/kaggle/input/tamil-labelled-dataset/Dataset with label/test/test.csv"
    test_images = "/kaggle/input/tamil-labelled-dataset/Dataset with label/test/images"

    # Load Data
    train_images, train_captions, train_labels = load_data(train_csv, train_images)
    test_images, test_captions, test_labels = load_data(test_csv, test_images)

    # Tokenizer and Models
    tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
    text_model = AutoModel.from_pretrained("ai4bharat/indic-bert").to(device)
    visual_model = models.resnet50(pretrained=True)
    visual_model = torch.nn.Sequential(*list(visual_model.children())[:-1]).to(device)

    # Transform for Images
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # Feature Extraction
    print("\nExtracting Text Features...")
    train_text_features = extract_text_features(train_captions, tokenizer, text_model)
    test_text_features = extract_text_features(test_captions, tokenizer, text_model)

    print("\nExtracting Image Features...")
    train_image_features = extract_image_features(train_images, visual_model, transform)
    test_image_features = extract_image_features(test_images, visual_model, transform)

    # Combine Features
    X_train = np.concatenate([train_text_features, train_image_features], axis=1)
    X_test = np.concatenate([test_text_features, test_image_features], axis=1)
    y_train = np.array(train_labels)
    y_test = np.array(test_labels)

    # Define Models
    classifiers = {
        "Logistic Regression": LogisticRegression(max_iter=1000),
        "Support Vector Machine": SVC(kernel="linear", probability=True),
        "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42)
    }

    # Train and Evaluate Models
    for model_name, model in classifiers.items():
        train_and_evaluate(X_train, y_train, X_test, y_test, model, model_name)

if __name__ == "__main__":
    main()


Using device: cuda


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



Extracting Text Features...


Extracting Text Features: 100%|██████████| 356/356 [00:02<00:00, 124.17it/s]



Extracting Image Features...


Extracting Image Features: 100%|██████████| 356/356 [00:07<00:00, 45.62it/s]



Training Logistic Regression...

Results for Logistic Regression:
              precision    recall  f1-score   support

           0       0.84      0.96      0.89       267
           1       0.76      0.44      0.56        89

    accuracy                           0.83       356
   macro avg       0.80      0.70      0.72       356
weighted avg       0.82      0.83      0.81       356

Accuracy: 0.8258426966292135
Macro F1 Score: 0.7243756243756244

Training Support Vector Machine...

Results for Support Vector Machine:
              precision    recall  f1-score   support

           0       0.84      0.91      0.87       267
           1       0.64      0.47      0.54        89

    accuracy                           0.80       356
   macro avg       0.74      0.69      0.71       356
weighted avg       0.79      0.80      0.79       356

Accuracy: 0.800561797752809
Macro F1 Score: 0.7072334510916778

Training Random Forest...

Results for Random Forest:
              precision 

In [54]:
# Class-wise metrics for each model
# Replace these with your actual values
results = {
    "Logistic Regression": {
        "precision_class_0": 0.84,
        "recall_class_0": 0.96,
        "f1_class_0": 0.89,
        "precision_class_1": 0.76,
        "recall_class_1": 0.44,
        "f1_class_1": 0.56
    },
    "Support Vector Machine": {
        "precision_class_0": 0.84,
        "recall_class_0": 0.91,
        "f1_class_0": 0.87,
        "precision_class_1": 0.64,
        "recall_class_1": 0.47,
        "f1_class_1": 0.54
    },
    "Random Forest": {
        "precision_class_0": 0.80,
        "recall_class_0": 0.99,
        "f1_class_0": 0.88,
        "precision_class_1": 0.91,
        "recall_class_1": 0.24,
        "f1_class_1": 0.38
    }
}

# Function to calculate macro-averaged metrics
def calculate_macro_metrics(metrics):
    macro_precision = (metrics["precision_class_0"] + metrics["precision_class_1"]) / 2
    macro_recall = (metrics["recall_class_0"] + metrics["recall_class_1"]) / 2
    macro_f1 = (metrics["f1_class_0"] + metrics["f1_class_1"]) / 2
    return macro_precision, macro_recall, macro_f1

# Calculate and print macro-averaged metrics for each model
for model_name, metrics in results.items():
    macro_precision, macro_recall, macro_f1 = calculate_macro_metrics(metrics)
    print(f"{model_name}:")
    print(f"  Macro Precision: {macro_precision:.4f}")
    print(f"  Macro Recall: {macro_recall:.4f}")
    print(f"  Macro F1 Score: {macro_f1:.4f}")
    print()


Logistic Regression:
  Macro Precision: 0.8000
  Macro Recall: 0.7000
  Macro F1 Score: 0.7250

Support Vector Machine:
  Macro Precision: 0.7400
  Macro Recall: 0.6900
  Macro F1 Score: 0.7050

Random Forest:
  Macro Precision: 0.8550
  Macro Recall: 0.6150
  Macro F1 Score: 0.6300



In [52]:
# Install required libraries
!pip install torchvision transformers

# Import libraries
import os
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from torchvision import models, transforms
from PIL import Image

# Device Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Paths
train_csv_path = "/kaggle/input/tamil-labelled-dataset/Dataset with label/train/train.csv"
train_images_path = "/kaggle/input/tamil-labelled-dataset/Dataset with label/train/images"
dev_csv_path = "/kaggle/input/tamil-labelled-dataset/Dataset with label/dev/dev.csv"
dev_images_path = "/kaggle/input/tamil-labelled-dataset/Dataset with label/dev/images"
test_csv_path = "/kaggle/input/tamil-labelled-dataset/Dataset with label/test/test.csv"
test_images_path = "/kaggle/input/tamil-labelled-dataset/Dataset with label/test/images"

# Load and Preprocess Dataset
def load_data(csv_path, image_dir):
    data = pd.read_csv(csv_path)
    images = []
    captions = []
    labels = []

    for idx in range(len(data)):
        img_name = str(data.loc[idx, 'image_id'])
        if not img_name.endswith(".jpg"):
            img_name += ".jpg"
        img_path = os.path.join(image_dir, img_name)

        if os.path.exists(img_path):
            images.append(img_path)
            captions.append(data.loc[idx, 'transcriptions'])
            if 'labels' in data.columns:
                labels.append(int(data.loc[idx, 'labels']))

    return images, captions, labels

# Feature Extraction Functions
def extract_text_features(captions, tokenizer, text_model, max_len=128):
    text_features = []
    for caption in tqdm(captions, desc="Extracting Text Features"):
        inputs = tokenizer(
            caption, return_tensors="pt", truncation=True, padding="max_length", max_length=max_len
        ).to(device)
        with torch.no_grad():
            outputs = text_model(**inputs)
            text_features.append(outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy())
    return np.array(text_features)

def extract_image_features(image_paths, visual_model, transform):
    image_features = []
    visual_model.eval()  # Ensure the model is in evaluation mode
    for img_path in tqdm(image_paths, desc="Extracting Image Features"):
        image = Image.open(img_path).convert("RGB")
        image = transform(image).unsqueeze(0).to(device)
        with torch.no_grad():
            features = visual_model(image).squeeze().cpu().numpy()
        image_features.append(features)
    return np.array(image_features)

# Evaluation Function
def evaluate_model(y_true, y_pred, model_name):
    print(f"\nResults for {model_name}:")
    print(classification_report(y_true, y_pred))
    print("Accuracy:", accuracy_score(y_true, y_pred))
    print("Macro Precision:", precision_score(y_true, y_pred, average="macro"))
    print("Macro Recall:", recall_score(y_true, y_pred, average="macro"))
    print("Macro F1 Score:", f1_score(y_true, y_pred, average="macro"))

# CNN Model
class CNNModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.fc1 = nn.Linear(128 * (input_dim // 2), 64)
        self.fc2 = nn.Linear(64, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add a dummy channel dimension: (batch_size, 1, input_dim)
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# BiLSTM Model
class BiLSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(BiLSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add a dummy sequence dimension: (batch_size, 1, input_dim)
        h_lstm, _ = self.lstm(x)
        out = self.fc(h_lstm[:, -1, :])  # Use the last hidden state
        return out

# BiLSTM + CNN Model
class BiLSTMCNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(BiLSTMCNNModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, bidirectional=True, batch_first=True)
        self.conv1 = nn.Conv1d(in_channels=hidden_dim * 2, out_channels=128, kernel_size=3, padding=1)
        self.fc = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        # Debugging: Print input shape
        print(f"Input shape: {x.shape}")

        if len(x.shape) == 2:  # (batch_size, input_dim)
            x = x.unsqueeze(1)  # (batch_size, 1, input_dim)

        # LSTM processing
        h_lstm, _ = self.lstm(x)  # (batch_size, sequence_length, hidden_dim * 2)
        print(f"LSTM output shape: {h_lstm.shape}")

        h_lstm = h_lstm.permute(0, 2, 1)  # (batch_size, hidden_dim * 2, sequence_length)
        print(f"Permuted LSTM output shape: {h_lstm.shape}")

        # Convolution
        h_conv = nn.functional.relu(self.conv1(h_lstm))  # (batch_size, 128, sequence_length)
        print(f"Conv1d output shape: {h_conv.shape}")

        # Global average pooling
        h_pool = torch.mean(h_conv, dim=2)  # (batch_size, 128)
        print(f"Global average pooling output shape: {h_pool.shape}")

        # Fully connected layer
        out = self.dropout(self.fc(h_pool))  # (batch_size, num_classes)
        print(f"Final output shape: {out.shape}")

        return out
        
# Training and Evaluation Function for CNN
def train_and_evaluate_cnn(X_train, y_train, X_dev, y_dev, X_test, y_test, input_dim, num_classes, epochs=10, batch_size=32):
    model = CNNModel(input_dim, num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

    # Convert data to PyTorch tensors
    X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train = torch.tensor(y_train, dtype=torch.long).to(device)
    X_dev = torch.tensor(X_dev, dtype=torch.float32).to(device)
    y_dev = torch.tensor(y_dev, dtype=torch.long).to(device)
    X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_test = torch.tensor(y_test, dtype=torch.long).to(device)

    # DataLoader
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training
    for epoch in range(epochs):
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        scheduler.step()

        # Validation
        model.eval()
        with torch.no_grad():
            outputs = model(X_dev)
            _, preds = torch.max(outputs, 1)
            val_loss = criterion(outputs, y_dev)
            print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}, Val Loss: {val_loss.item()}")

    # Evaluation on Test Data
    model.eval()
    with torch.no_grad():
        outputs = model(X_test)
        _, preds = torch.max(outputs, 1)
        evaluate_model(y_test.cpu(), preds.cpu(), "CNN")

# Training and Evaluation Function for BiLSTM
def train_and_evaluate_bilstm(X_train, y_train, X_dev, y_dev, X_test, y_test, input_dim, hidden_dim, num_classes, epochs=10, batch_size=32):
    model = BiLSTMModel(input_dim, hidden_dim, num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

    # Convert data to PyTorch tensors
    X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train = torch.tensor(y_train, dtype=torch.long).to(device)
    X_dev = torch.tensor(X_dev, dtype=torch.float32).to(device)
    y_dev = torch.tensor(y_dev, dtype=torch.long).to(device)
    X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_test = torch.tensor(y_test, dtype=torch.long).to(device)

    # DataLoader
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training
    for epoch in range(epochs):
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        scheduler.step()

        # Validation
        model.eval()
        with torch.no_grad():
            outputs = model(X_dev)
            _, preds = torch.max(outputs, 1)
            val_loss = criterion(outputs, y_dev)
            print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}, Val Loss: {val_loss.item()}")

    # Evaluation on Test Data
    model.eval()
    with torch.no_grad():
        outputs = model(X_test)
        _, preds = torch.max(outputs, 1)
        evaluate_model(y_test.cpu(), preds.cpu(), "BiLSTM")

# Training and Evaluation Function for BiLSTM + CNN
def train_and_evaluate_bilstm_cnn(X_train, y_train, X_dev, y_dev, X_test, y_test, input_dim, hidden_dim, num_classes, epochs=10, batch_size=32):
    model = BiLSTMCNNModel(input_dim, hidden_dim, num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

    # Convert data to PyTorch tensors
    X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train = torch.tensor(y_train, dtype=torch.long).to(device)
    X_dev = torch.tensor(X_dev, dtype=torch.float32).to(device)
    y_dev = torch.tensor(y_dev, dtype=torch.long).to(device)
    X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_test = torch.tensor(y_test, dtype=torch.long).to(device)

    # DataLoader
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training
    for epoch in range(epochs):
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        scheduler.step()

        # Validation
        model.eval()
        with torch.no_grad():
            outputs = model(X_dev)
            _, preds = torch.max(outputs, 1)
            val_loss = criterion(outputs, y_dev)
            print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}, Val Loss: {val_loss.item()}")

    # Evaluation on Test Data
    model.eval()
    with torch.no_grad():
        outputs = model(X_test)
        _, preds = torch.max(outputs, 1)
        evaluate_model(y_test.cpu(), preds.cpu(), "BiLSTM + CNN")
        
# Main Function
def main():
    # Load Data
    train_image_paths, train_captions, train_labels = load_data(train_csv_path, train_images_path)
    dev_image_paths, dev_captions, dev_labels = load_data(dev_csv_path, dev_images_path)
    test_image_paths, test_captions, test_labels = load_data(test_csv_path, test_images_path)

    # Tokenizer and Models
    tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
    text_model = AutoModel.from_pretrained("ai4bharat/indic-bert").to(device)
    visual_model = models.resnet50(pretrained=True)
    visual_model = torch.nn.Sequential(*list(visual_model.children())[:-1]).to(device)

    # Transform for Images
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # Feature Extraction
    print("\nExtracting Text Features...")
    train_text_features = extract_text_features(train_captions, tokenizer, text_model)
    dev_text_features = extract_text_features(dev_captions, tokenizer, text_model)
    test_text_features = extract_text_features(test_captions, tokenizer, text_model)

    print("\nExtracting Image Features...")
    train_image_features = extract_image_features(train_image_paths, visual_model, transform)
    dev_image_features = extract_image_features(dev_image_paths, visual_model, transform)
    test_image_features = extract_image_features(test_image_paths, visual_model, transform)

    # Combine Features
    X_train = np.concatenate([train_text_features, train_image_features], axis=1)
    X_dev = np.concatenate([dev_text_features, dev_image_features], axis=1)
    X_test = np.concatenate([test_text_features, test_image_features], axis=1)
    y_train = np.array(train_labels)
    y_dev = np.array(dev_labels)
    y_test = np.array(test_labels)

    # Train and Evaluate Models
    input_dim = X_train.shape[1]
    num_classes = 2
    hidden_dim = 128

    print("\nTraining CNN Model...")
    train_and_evaluate_cnn(X_train, y_train, X_dev, y_dev, X_test, y_test, input_dim, num_classes)

    print("\nTraining BiLSTM Model...")
    train_and_evaluate_bilstm(X_train, y_train, X_dev, y_dev, X_test, y_test, input_dim, hidden_dim, num_classes)

    print("\nTraining BiLSTM + CNN Model...")
    train_and_evaluate_bilstm_cnn(X_train, y_train, X_dev, y_dev, X_test, y_test, input_dim, hidden_dim, num_classes)

if __name__ == "__main__":
    main()

Using device: cuda


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



Extracting Text Features...


Extracting Text Features: 100%|██████████| 356/356 [00:02<00:00, 123.19it/s]



Extracting Image Features...


Extracting Image Features: 100%|██████████| 356/356 [00:07<00:00, 50.15it/s]



Training CNN Model...
Epoch 1/10, Loss: 0.574783205986023, Val Loss: 0.5768593549728394
Epoch 2/10, Loss: 0.6249182224273682, Val Loss: 0.599402904510498
Epoch 3/10, Loss: 0.5841427445411682, Val Loss: 0.533103883266449
Epoch 4/10, Loss: 0.604602038860321, Val Loss: 0.5732274651527405
Epoch 5/10, Loss: 0.4042581021785736, Val Loss: 0.4958321452140808
Epoch 6/10, Loss: 0.4252728521823883, Val Loss: 0.4980243742465973
Epoch 7/10, Loss: 0.34447580575942993, Val Loss: 0.4950961172580719
Epoch 8/10, Loss: 0.31021648645401, Val Loss: 0.49474743008613586
Epoch 9/10, Loss: 0.4441564083099365, Val Loss: 0.4926066994667053
Epoch 10/10, Loss: 0.6073155999183655, Val Loss: 0.4924260973930359

Results for CNN:
              precision    recall  f1-score   support

           0       0.75      1.00      0.86       267
           1       0.00      0.00      0.00        89

    accuracy                           0.75       356
   macro avg       0.38      0.50      0.43       356
weighted avg       0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

Epoch 2/10, Loss: 0.4890260398387909, Val Loss: 0.462737500667572
Epoch 3/10, Loss: 0.40326637029647827, Val Loss: 0.4342435300350189
Epoch 4/10, Loss: 0.2953743636608124, Val Loss: 0.42356517910957336
Epoch 5/10, Loss: 0.29275739192962646, Val Loss: 0.4269227683544159
Epoch 6/10, Loss: 0.28263798356056213, Val Loss: 0.42825138568878174
Epoch 7/10, Loss: 0.30954357981681824, Val Loss: 0.43716540932655334
Epoch 8/10, Loss: 0.15827812254428864, Val Loss: 0.43220025300979614
Epoch 9/10, Loss: 0.14452527463436127, Val Loss: 0.42693421244621277
Epoch 10/10, Loss: 0.1946982443332672, Val Loss: 0.4273425340652466

Results for BiLSTM:
              precision    recall  f1-score   support

           0       0.86      0.92      0.89       267
           1       0.70      0.56      0.62        89

    accuracy                           0.83       356
   macro avg       0.78      0.74      0.76       356
weighted avg       0.82      0.83      0.82       356

Accuracy: 0.8314606741573034
Macro Pre